# The impact of the price of LPG for the needy population in relation to the minimum wage. An analysis from 2013 to 2023.

## IMPORTING LIBRARIES

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import re
import pandas as pd
import numpy as np
import tabula
import os

## DATA EXTRACTION

In [2]:
# WEB SCRAPPING TABLE (df_salario):

service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service = service)
driver.get('https://portal.trt3.jus.br/internet/servicos/valores/salario-minimo')
url = driver.find_element(By.XPATH,'//*[@id="parent-fieldname-text"]/div/table')
html = url.get_attribute('outerHTML')
driver.quit()
df_salario = pd.read_html(html)[0]

In [3]:
# WEB SCRAPPING TABLE FROM PDF (margens_2013):

margens_2013 = tabula.read_pdf(r'C:\Users\pedro\OneDrive\Área de Trabalho\GAS\margens_2013.pdf', pages = 'all')
margens_13 = pd.concat(margens_2013)

In [4]:
# WEB SCRAPPING TABLE FROM PDF (margens_2023):

margens = tabula.read_pdf(r'C:\Users\pedro\OneDrive\Área de Trabalho\GAS\margens_2023.pdf', pages = 'all')
margens_23 = pd.concat(margens)

In [5]:
#READING XLSX FILES IN DATAFRAME (brasil):

brasil = pd.read_excel(r'C:\Users\pedro\OneDrive\Área de Trabalho\GAS\brasil_semanal.xlsx')

In [6]:
#READING XLSX FILES IN DATAFRAME (estados):

estados = pd.read_excel(r'C:\Users\pedro\OneDrive\Área de Trabalho\GAS\estados_semanal.xlsx')

In [7]:
#READING XLSX FILES IN DATAFRAME COMPLEMENT STATES ICMS ()
to = pd.read_excel(r'C:\Users\pedro\OneDrive\Área de Trabalho\GAS\TO.xlsx')
sc = pd.read_excel(r'C:\Users\pedro\OneDrive\Área de Trabalho\GAS\SC.xlsx')
rn = pd.read_excel(r'C:\Users\pedro\OneDrive\Área de Trabalho\GAS\RN.xlsx')
pr = pd.read_excel(r'C:\Users\pedro\OneDrive\Área de Trabalho\GAS\PR.xlsx')
ms = pd.read_excel(r'C:\Users\pedro\OneDrive\Área de Trabalho\GAS\MS.xlsx')
es = pd.read_excel(r'C:\Users\pedro\OneDrive\Área de Trabalho\GAS\ES.xlsx')
am = pd.read_excel(r'C:\Users\pedro\OneDrive\Área de Trabalho\GAS\AM.xlsx')

complemento_icms = pd.concat([to,sc,rn,pr,ms,es,am])

## EXPLORING AND CLEANING THE DATA

### DATAFRAME df_salario:

In [8]:
#PERFORMING TREATMENT IN DATAFRAME df_salario USING DE FUNCTION BELOW:
    
def limpar_df_salario(df):
    
    '''THIS FUNCTION PERFORMS THE CLEANING TREATMENT ON THE DATAFRAME 'df_salario'.'''
    
    df = df.drop(columns = ['MOEDA','FONTE']).drop(index=range(12,134))
    df = df.rename(columns = {'ANO/VIGÊNCIA':'DATA', 'VALOR':'VALOR POR MÊS'})
    df['VALOR POR MÊS'] = df['VALOR POR MÊS'].str.slice(stop=11)
    df['VALOR POR MÊS'] = df['VALOR POR MÊS'].str.replace('P','').str.replace(',','.')
    df['VALOR POR MÊS'] = df['VALOR POR MÊS'].str.replace(r'R\$ +', '', regex=True)
    df['VALOR POR MÊS'] = df['VALOR POR MÊS'].str.replace(r'R\$\xa0', '', regex=True)
    df['VALOR POR MÊS'] = df['VALOR POR MÊS'].str.replace(r'\xa0', '', regex=True)
    df['DATA'] = df['DATA'].str.replace(r'\s+\d{1,2}º/\d{1,2}/\d{4}', '', regex=True).str.replace('- ','')
    df['DATA'] = pd.to_datetime('01- Jan'+df['DATA'])
    lista = ['1302,00', '1212,00', '1100,00', '1045,00', '1039,00',
       '998,00', '954,00', '937,00', '880,00', '788,00', '724,00',
       '678,00']
    df['VALOR POR MÊS'] = lista
    return df
    

In [9]:
# NEW DATAFRAME TREATED (salario_minimo):

salario_minimo = limpar_df_salario(df_salario)


### DATAFRAME margens_2013

In [10]:
#PERFORMING TREATMENT IN DATAFRAME margens_13 USING DE FUNCTION BELOW:

def brasil_icms(margens_13):
    
    '''This function performs the following steps on any project dataframe: cleans columns and rows that are not relevant to the analysis;
    performs substitutions in order to improve aesthetics and finally filters the results only from GLP.'''
    
    estado = margens_13.drop(columns = {'Obs: A partir de abril/04, passou a vigorar a nova sistemática de cobrança de ICMS de GLP em 11 estados (AM, AL, BA, CE, MA, RN, PA, PB, PE, RJ e SE), que consideram que parte do\rproduto é produzido a partir do gás natural.\rA parcela referente à bi-tributação de ICMS nestes estados está incluída na Margem Bruta de Distribuição.\r(1) A Resolução G SEFAZ n.o 13/08, de 22/09/08, incluiu o GLP acondicionado em recipientes transportáveis com capacidade de até 13 kg, destinado ao consumo doméstico, na lista\rde mercadorias integrantes da cesta básica. Dessa forma, sua alíquota de ICMS caiu de 17% para 1%, conforme Resolução G SEFAZ n.o 11/08, de 14/08/08. A Lei Estadual n.o 3.361,\rde 30/12/2008, em vigor a partir de 1o de janeiro de 2009, isentou do ICMS as operações internas com GLP destinado ao consumo doméstico e acondicionado em recipientes\rtransportáveis com capacidade de até 13 kg. A alínea "b" da Lei Complementar Estadual no 112, de 21/12/2012, em vigor desde 1/4/2013, alterou para 17% o valor da alíquota de\rICMS das operações internas com GLP.\r(2) Não foi possível calcular as margens brutas de distribuição e de revenda em função da ausência de preços de distribuição.\rNotas:\rPreços de Produtores:\rDesde janeiro de 2002, preços informados, semanalmente, pelos produtores e importadores, nos termos da Portaria ANP n.o 297, de 18/12/2001.\rEm contrapartida à retirada do subsídio aos preços de GLP, foi instituído, por meio do Decreto n.o 4.102, de 24/01/2002, o programa "Auxílio-Gás" no valor de R$ 7,50 / mês para\rfamílias de baixa renda, posteriormente unificado pelo programa "Bolsa-Família" por meio da Lei n.o 10.836, de 09 de janeiro de 2004.\rOs preços referem-se às vendas de GLP por botijão de 13 kg.\rICMS:\rAs alíquotas, as margens de valor agregado e os preços médios ponderados ao consumidor final (PMPF) são estabelecidos por Convênios CONFAZ e Atos COTEPE e variam de estado\rpara estado.\rPara formação do preço Brasil, o ICMS foi calculado com base nos valores de cada estado, ponderado pelos  respectivos volumes de comercialização.\rMargens de Distribuição e de Revenda:\rCalculadas com base nos dados semanais do Levantamento de Preços e de Margens de Comercialização de Combustíveis da ANP, regulamentado pela Portaria ANP n.o 202, de\r15/08/2000.\rOs preços de distribuição podem, eventualmente, contemplar valores relativos à descontos por pagamentos de duplicatas no vencimento e por cumprimento de metas de vendas.'})
    estado = estado.drop(columns = {'Unnamed: 6','Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5'})
    estado = estado.reset_index(drop=True).drop(index=[0,2,3,4,6,7,8,9,10,11,12])
    estado = estado.rename(columns = {'Unnamed: 0':'ESTADO','Unnamed: 1':'ICMS'	})
    estado = estado.iloc[0:2]
    estado = estado.reset_index(drop=True).drop(index=[0])
    estado.fillna('BRASIL', inplace=True)
    
    return estado



In [11]:
brasil_icms_13 = brasil_icms(margens_13)

### DATAFRAME brasil

In [12]:
#PERFORMING TREATMENT IN DATAFRAME brasil USING DE FUNCTION BELOW:

def limpar_df_br(df):
    '''This function performs the following steps on any project dataframe: cleans columns and rows that are not relevant to the analysis;
    performs substitutions in order to improve aesthetics and finally filters the results only from GLP.'''
    import pandas as pd
    import numpy as np
    from datetime import datetime
    df = df.rename(columns = {'AGÊNCIA NACIONAL DO PETRÓLEO, GÁS NATURAL E BIOCOMBUSTÍVEIS - ANP':'DATA INICIAL','Unnamed: 1':'DATA FINAL', 'Unnamed: 2':'PRODUTO','Unnamed: 3':'NÚMERO DE POSTOS PESQUISADOS','Unnamed: 4':'UNIDADE DE MEDIDA','Unnamed: 5':'PREÇO MÉDIO REVENDA','Unnamed: 6':'DESVIO PADRÃO REVENDA','Unnamed: 7':'PREÇO MÍNIMO REVENDA','Unnamed: 8':'PREÇO MÁXIMO REVENDA','Unnamed: 9':'MARGEM MÉDIA REVENDA','Unnamed: 10':'COEF DE VARIAÇÃO REVENDA','Unnamed: 11':'PREÇO MÉDIO DISTRIBUIÇÃO','Unnamed: 12':'DESVIO PADRÃO DISTRIBUIÇÃO','Unnamed: 13':'PREÇO MÍNIMO DISTRIBUIÇÃO','Unnamed: 14':'PREÇO MÁXIMO DISTRIBUIÇÃO','Unnamed: 15':'COEF DE VARIAÇÃO DISTRIBUIÇÃO'})
    df_1 = df.drop(index=range(0, 17)).drop(columns=['DATA INICIAL', 'DESVIO PADRÃO REVENDA', 'MARGEM MÉDIA REVENDA','COEF DE VARIAÇÃO REVENDA', 'PREÇO MÉDIO DISTRIBUIÇÃO','DESVIO PADRÃO DISTRIBUIÇÃO',	'PREÇO MÍNIMO DISTRIBUIÇÃO','PREÇO MÁXIMO DISTRIBUIÇÃO','COEF DE VARIAÇÃO DISTRIBUIÇÃO'])
    df_limpo = df_1.loc[df_1['PRODUTO'] == 'GLP']
    df_limpo = df_limpo.replace('-', 'Não Informado').replace('R$/13Kg', '13Kg').replace('R$/13kg', '13Kg')
    df_limpo['DATA FINAL'] = pd.to_datetime(df_limpo['DATA FINAL'])
    df_limpo['DATA FINAL'] = df_limpo['DATA FINAL'].apply(lambda x: datetime.strftime(x, '%b-%Y'))
    df_limpo = df_limpo.set_index('DATA FINAL')
    df_limpo = df_limpo.reset_index()
    df_limpo = df_limpo.rename(columns = {'DATA FINAL': 'DATA'})
    df_limpo['DATA'] = pd.to_datetime('01-'+df_limpo['DATA'])
    
    
    return df_limpo


In [46]:
# NEW DATAFRAME TREATED (glp_br):
glp_br = limpar_df_br(brasil)


### DATAFRAME estado

In [14]:
#PERFORMING TREATMENT IN DATAFRAME estados USING DE FUNCTION BELOW:

def limpar_df_estado(df):
    '''This function performs the following steps on any project dataframe: cleans columns and rows that are not relevant to the analysis;
    performs substitutions in order to improve aesthetics and finally filters the results only from GLP.'''
    import pandas as pd
    import numpy as np
    from datetime import datetime   
    df = df.drop(index=range(0, 17)).drop(columns = ['AGÊNCIA NACIONAL DO PETRÓLEO, GÁS NATURAL E BIOCOMBUSTÍVEIS - ANP', 'Unnamed: 8', 'Unnamed: 11','Unnamed: 12','Unnamed: 13','Unnamed: 14','Unnamed: 15','Unnamed: 16','Unnamed: 17'])
    estados_limpo = df.rename(columns = {'Unnamed: 1':'DATA FINAL', 'Unnamed: 2':'REGIÃO','Unnamed: 3':'ESTADO','Unnamed: 4':'PRODUTO','Unnamed: 5':'NÚMERO DE POSTOS PESQUISADOS','Unnamed: 6':'UNIDADE DE MEDIDA','Unnamed: 7':'PREÇO MÉDIO REVENDA','Unnamed: 9':'PREÇO MÍNIMO REVENDA','Unnamed: 10':'PREÇO MÁXIMO REVENDA'})
    df_limpo = estados_limpo.loc[estados_limpo['PRODUTO'] == 'GLP']
    df_limpo = df_limpo.replace('-', 'Não Informado').replace('R$/13Kg', '13Kg').replace('R$/13kg', '13Kg')
    df_limpo['DATA FINAL'] = pd.to_datetime(df_limpo['DATA FINAL'])
    df_limpo['DATA FINAL'] = df_limpo['DATA FINAL'].apply(lambda x: datetime.strftime(x, '%b-%Y'))
    df_limpo = df_limpo.set_index('DATA FINAL')
    df_limpo.index.name = 'DATA'
    df_limpo = df_limpo.reset_index()
    df_limpo['DATA'] = pd.to_datetime('01-'+df_limpo['DATA'])
    
    return df_limpo


In [45]:
# NEW DATAFRAME TREATED (glp_regioes):
glp_regioes = limpar_df_estado(estados)

### DATAFRAME margens_13

In [16]:
# DEFINING THE FUNCTION THAT EXTRACTS BRAZIL ONLY FROM THE GROUP:
def brasil_icms(margens_13, estado):
    
    '''THIS FUNCTION CARRIES OUT THE CLEANING TREATMENT OF THE 'margens_13' DATAFRAME, AIMED AT RETURNING GENERAL DATA FOR BRAZIL.'''
    
    df = margens_13.drop(columns = {'Obs: A partir de abril/04, passou a vigorar a nova sistemática de cobrança de ICMS de GLP em 11 estados (AM, AL, BA, CE, MA, RN, PA, PB, PE, RJ e SE), que consideram que parte do\rproduto é produzido a partir do gás natural.\rA parcela referente à bi-tributação de ICMS nestes estados está incluída na Margem Bruta de Distribuição.\r(1) A Resolução G SEFAZ n.o 13/08, de 22/09/08, incluiu o GLP acondicionado em recipientes transportáveis com capacidade de até 13 kg, destinado ao consumo doméstico, na lista\rde mercadorias integrantes da cesta básica. Dessa forma, sua alíquota de ICMS caiu de 17% para 1%, conforme Resolução G SEFAZ n.o 11/08, de 14/08/08. A Lei Estadual n.o 3.361,\rde 30/12/2008, em vigor a partir de 1o de janeiro de 2009, isentou do ICMS as operações internas com GLP destinado ao consumo doméstico e acondicionado em recipientes\rtransportáveis com capacidade de até 13 kg. A alínea "b" da Lei Complementar Estadual no 112, de 21/12/2012, em vigor desde 1/4/2013, alterou para 17% o valor da alíquota de\rICMS das operações internas com GLP.\r(2) Não foi possível calcular as margens brutas de distribuição e de revenda em função da ausência de preços de distribuição.\rNotas:\rPreços de Produtores:\rDesde janeiro de 2002, preços informados, semanalmente, pelos produtores e importadores, nos termos da Portaria ANP n.o 297, de 18/12/2001.\rEm contrapartida à retirada do subsídio aos preços de GLP, foi instituído, por meio do Decreto n.o 4.102, de 24/01/2002, o programa "Auxílio-Gás" no valor de R$ 7,50 / mês para\rfamílias de baixa renda, posteriormente unificado pelo programa "Bolsa-Família" por meio da Lei n.o 10.836, de 09 de janeiro de 2004.\rOs preços referem-se às vendas de GLP por botijão de 13 kg.\rICMS:\rAs alíquotas, as margens de valor agregado e os preços médios ponderados ao consumidor final (PMPF) são estabelecidos por Convênios CONFAZ e Atos COTEPE e variam de estado\rpara estado.\rPara formação do preço Brasil, o ICMS foi calculado com base nos valores de cada estado, ponderado pelos  respectivos volumes de comercialização.\rMargens de Distribuição e de Revenda:\rCalculadas com base nos dados semanais do Levantamento de Preços e de Margens de Comercialização de Combustíveis da ANP, regulamentado pela Portaria ANP n.o 202, de\r15/08/2000.\rOs preços de distribuição podem, eventualmente, contemplar valores relativos à descontos por pagamentos de duplicatas no vencimento e por cumprimento de metas de vendas.'})
    df = df.drop(columns = {'Unnamed: 6','Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5'})
    df = df.reset_index(drop=True).drop(index=[0,2,3,4,6,7,8,9,10,11,12])
    df = df.replace('ICMS', '-')
    df = df.rename(columns = {'Unnamed: 0':'ESTADO','Unnamed: 1':'ICMS em %'	})
    df = df.iloc[0:2]
    df = df.reset_index(drop=True).drop(index=[0])
    df.fillna(estado, inplace=True)
    df = df.replace('ICMS', '-')
    df = df.set_index('ESTADO')
    
    return df

In [17]:
 # NEW DATAFRAME TREATED (brasil_icms_13):
 
 brasil_icms_13 = brasil_icms(margens_13, 'BRASIL' ).reset_index()

In [18]:
# DEFINING THE FUNCTION FOR THE STATE THAT DO NOT REQUIRE SPECIAL TREATMENT IN THE MAIN DATAFRAME:
def acre_icms_13(margens_13, estado):
    
    '''THIS FUNCTION CARRIES OUT THE CLEANING TREATMENT OF THE DATAFRAME 'margens_13', AIMED AT RETURNING GENERAL DATA TO THE STATE DEFINED IN THE FUNCTION ARGUMENT.'''
    
    df = margens_13.drop(columns = {'Obs: A partir de abril/04, passou a vigorar a nova sistemática de cobrança de ICMS de GLP em 11 estados (AM, AL, BA, CE, MA, RN, PA, PB, PE, RJ e SE), que consideram que parte do\rproduto é produzido a partir do gás natural.\rA parcela referente à bi-tributação de ICMS nestes estados está incluída na Margem Bruta de Distribuição.\r(1) A Resolução G SEFAZ n.o 13/08, de 22/09/08, incluiu o GLP acondicionado em recipientes transportáveis com capacidade de até 13 kg, destinado ao consumo doméstico, na lista\rde mercadorias integrantes da cesta básica. Dessa forma, sua alíquota de ICMS caiu de 17% para 1%, conforme Resolução G SEFAZ n.o 11/08, de 14/08/08. A Lei Estadual n.o 3.361,\rde 30/12/2008, em vigor a partir de 1o de janeiro de 2009, isentou do ICMS as operações internas com GLP destinado ao consumo doméstico e acondicionado em recipientes\rtransportáveis com capacidade de até 13 kg. A alínea "b" da Lei Complementar Estadual no 112, de 21/12/2012, em vigor desde 1/4/2013, alterou para 17% o valor da alíquota de\rICMS das operações internas com GLP.\r(2) Não foi possível calcular as margens brutas de distribuição e de revenda em função da ausência de preços de distribuição.\rNotas:\rPreços de Produtores:\rDesde janeiro de 2002, preços informados, semanalmente, pelos produtores e importadores, nos termos da Portaria ANP n.o 297, de 18/12/2001.\rEm contrapartida à retirada do subsídio aos preços de GLP, foi instituído, por meio do Decreto n.o 4.102, de 24/01/2002, o programa "Auxílio-Gás" no valor de R$ 7,50 / mês para\rfamílias de baixa renda, posteriormente unificado pelo programa "Bolsa-Família" por meio da Lei n.o 10.836, de 09 de janeiro de 2004.\rOs preços referem-se às vendas de GLP por botijão de 13 kg.\rICMS:\rAs alíquotas, as margens de valor agregado e os preços médios ponderados ao consumidor final (PMPF) são estabelecidos por Convênios CONFAZ e Atos COTEPE e variam de estado\rpara estado.\rPara formação do preço Brasil, o ICMS foi calculado com base nos valores de cada estado, ponderado pelos  respectivos volumes de comercialização.\rMargens de Distribuição e de Revenda:\rCalculadas com base nos dados semanais do Levantamento de Preços e de Margens de Comercialização de Combustíveis da ANP, regulamentado pela Portaria ANP n.o 202, de\r15/08/2000.\rOs preços de distribuição podem, eventualmente, contemplar valores relativos à descontos por pagamentos de duplicatas no vencimento e por cumprimento de metas de vendas.'})
    df = df.drop(columns = {'Unnamed: 6','Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5'})
    df = df.reset_index(drop=True).drop(index=[0,2,3,4,6,7,8,9,10,11,12])
    df = df.rename(columns = {'Unnamed: 0':'ESTADO','Unnamed: 1':'ICMS'	})
    df = df.reset_index(drop=True)
    df = df.iloc[2:12]
    df = df.drop(index = [2,3,4,6,7,8,9,10,11])
    df= df.rename(columns = {'ICMS':'ICMS em %'})
    df = df.replace('17%', estado).replace('ICMS','17%')
    df = df.set_index('ESTADO')
    
    return df    

In [19]:
# NEW DATAFRAME TREATED (ac_13):

ac_13 = acre_icms_13(margens_13, 'AC')

In [20]:
# DEFINING THE FUNCTION FOR THE STATE THAT DO NOT REQUIRE SPECIAL TREATMENT IN THE MAIN DATAFRAME:

def alagoas_icms_13(margens_13, estado): 
    
    '''THIS FUNCTION CARRIES OUT THE CLEANING TREATMENT OF THE DATAFRAME 'margens_13', AIMED AT RETURNING GENERAL DATA TO THE STATE DEFINED IN THE FUNCTION ARGUMENT.'''
    
    df = margens_13.drop(columns = {'Obs: A partir de abril/04, passou a vigorar a nova sistemática de cobrança de ICMS de GLP em 11 estados (AM, AL, BA, CE, MA, RN, PA, PB, PE, RJ e SE), que consideram que parte do\rproduto é produzido a partir do gás natural.\rA parcela referente à bi-tributação de ICMS nestes estados está incluída na Margem Bruta de Distribuição.\r(1) A Resolução G SEFAZ n.o 13/08, de 22/09/08, incluiu o GLP acondicionado em recipientes transportáveis com capacidade de até 13 kg, destinado ao consumo doméstico, na lista\rde mercadorias integrantes da cesta básica. Dessa forma, sua alíquota de ICMS caiu de 17% para 1%, conforme Resolução G SEFAZ n.o 11/08, de 14/08/08. A Lei Estadual n.o 3.361,\rde 30/12/2008, em vigor a partir de 1o de janeiro de 2009, isentou do ICMS as operações internas com GLP destinado ao consumo doméstico e acondicionado em recipientes\rtransportáveis com capacidade de até 13 kg. A alínea "b" da Lei Complementar Estadual no 112, de 21/12/2012, em vigor desde 1/4/2013, alterou para 17% o valor da alíquota de\rICMS das operações internas com GLP.\r(2) Não foi possível calcular as margens brutas de distribuição e de revenda em função da ausência de preços de distribuição.\rNotas:\rPreços de Produtores:\rDesde janeiro de 2002, preços informados, semanalmente, pelos produtores e importadores, nos termos da Portaria ANP n.o 297, de 18/12/2001.\rEm contrapartida à retirada do subsídio aos preços de GLP, foi instituído, por meio do Decreto n.o 4.102, de 24/01/2002, o programa "Auxílio-Gás" no valor de R$ 7,50 / mês para\rfamílias de baixa renda, posteriormente unificado pelo programa "Bolsa-Família" por meio da Lei n.o 10.836, de 09 de janeiro de 2004.\rOs preços referem-se às vendas de GLP por botijão de 13 kg.\rICMS:\rAs alíquotas, as margens de valor agregado e os preços médios ponderados ao consumidor final (PMPF) são estabelecidos por Convênios CONFAZ e Atos COTEPE e variam de estado\rpara estado.\rPara formação do preço Brasil, o ICMS foi calculado com base nos valores de cada estado, ponderado pelos  respectivos volumes de comercialização.\rMargens de Distribuição e de Revenda:\rCalculadas com base nos dados semanais do Levantamento de Preços e de Margens de Comercialização de Combustíveis da ANP, regulamentado pela Portaria ANP n.o 202, de\r15/08/2000.\rOs preços de distribuição podem, eventualmente, contemplar valores relativos à descontos por pagamentos de duplicatas no vencimento e por cumprimento de metas de vendas.'})
    df = df.drop(columns = {'Unnamed: 6','Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5'})
    df = df.reset_index(drop=True).drop(index=[0,2,3,4,6,7,8,9,10,11,12])
    df = df.rename(columns = {'Unnamed: 0':'ESTADO','Unnamed: 1':'ICMS'	})
    df = df.reset_index(drop=True)
    df = df.iloc[13:21]
    df = df.drop(index = [13,14,15,17,18,19,20])
    df= df.rename(columns = {'ICMS':'ICMS em %'})
    df = df.replace('17%', estado).replace('ICMS','17%')
    df = df.set_index('ESTADO')
   
    return df

In [21]:
# NEW DATAFRAME TREATED (al_13):
 
al_13 = alagoas_icms_13(margens_13,'AL')

In [22]:
# DEFINING THE FUNCTION FOR THE STATE THAT DO NOT REQUIRE SPECIAL TREATMENT IN THE MAIN DATAFRAME:

def ceara_icms_13(margens_13, estado): 
    
    '''THIS FUNCTION CARRIES OUT THE CLEANING TREATMENT OF THE DATAFRAME 'margens_13', AIMED AT RETURNING GENERAL DATA TO THE STATE DEFINED IN THE FUNCTION ARGUMENT.'''
    
    df = margens_13.drop(columns = {'Obs: A partir de abril/04, passou a vigorar a nova sistemática de cobrança de ICMS de GLP em 11 estados (AM, AL, BA, CE, MA, RN, PA, PB, PE, RJ e SE), que consideram que parte do\rproduto é produzido a partir do gás natural.\rA parcela referente à bi-tributação de ICMS nestes estados está incluída na Margem Bruta de Distribuição.\r(1) A Resolução G SEFAZ n.o 13/08, de 22/09/08, incluiu o GLP acondicionado em recipientes transportáveis com capacidade de até 13 kg, destinado ao consumo doméstico, na lista\rde mercadorias integrantes da cesta básica. Dessa forma, sua alíquota de ICMS caiu de 17% para 1%, conforme Resolução G SEFAZ n.o 11/08, de 14/08/08. A Lei Estadual n.o 3.361,\rde 30/12/2008, em vigor a partir de 1o de janeiro de 2009, isentou do ICMS as operações internas com GLP destinado ao consumo doméstico e acondicionado em recipientes\rtransportáveis com capacidade de até 13 kg. A alínea "b" da Lei Complementar Estadual no 112, de 21/12/2012, em vigor desde 1/4/2013, alterou para 17% o valor da alíquota de\rICMS das operações internas com GLP.\r(2) Não foi possível calcular as margens brutas de distribuição e de revenda em função da ausência de preços de distribuição.\rNotas:\rPreços de Produtores:\rDesde janeiro de 2002, preços informados, semanalmente, pelos produtores e importadores, nos termos da Portaria ANP n.o 297, de 18/12/2001.\rEm contrapartida à retirada do subsídio aos preços de GLP, foi instituído, por meio do Decreto n.o 4.102, de 24/01/2002, o programa "Auxílio-Gás" no valor de R$ 7,50 / mês para\rfamílias de baixa renda, posteriormente unificado pelo programa "Bolsa-Família" por meio da Lei n.o 10.836, de 09 de janeiro de 2004.\rOs preços referem-se às vendas de GLP por botijão de 13 kg.\rICMS:\rAs alíquotas, as margens de valor agregado e os preços médios ponderados ao consumidor final (PMPF) são estabelecidos por Convênios CONFAZ e Atos COTEPE e variam de estado\rpara estado.\rPara formação do preço Brasil, o ICMS foi calculado com base nos valores de cada estado, ponderado pelos  respectivos volumes de comercialização.\rMargens de Distribuição e de Revenda:\rCalculadas com base nos dados semanais do Levantamento de Preços e de Margens de Comercialização de Combustíveis da ANP, regulamentado pela Portaria ANP n.o 202, de\r15/08/2000.\rOs preços de distribuição podem, eventualmente, contemplar valores relativos à descontos por pagamentos de duplicatas no vencimento e por cumprimento de metas de vendas.'})
    df = df.drop(columns = {'Unnamed: 6','Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5'})
    df = df.reset_index(drop=True).drop(index=[0,2,3,4,6,7,8,9,10,11,12])
    df = df.rename(columns = {'Unnamed: 0':'ESTADO','Unnamed: 1':'ICMS'	})
    df = df.reset_index(drop=True)
    df = df.iloc[27:38]
    df = df.drop(index = [27,28,29,31,32,33,34,35,36,37])
    df= df.rename(columns = {'ICMS':'ICMS em %'})
    df = df.replace('17%', estado).replace('ICMS','17%')
    df = df.set_index('ESTADO')
    
    return df

In [23]:
# NEW DATAFRAME TREATED (ce_13):

ce_13 = ceara_icms_13(margens_13,'CE')

In [24]:
# DEFINING THE FUNCTION FOR THE STATE THAT DO NOT REQUIRE SPECIAL TREATMENT IN THE MAIN DATAFRAME:

def distrito_icms_13(margens_13, estado): 
    
    '''THIS FUNCTION CARRIES OUT THE CLEANING TREATMENT OF THE DATAFRAME 'margens_13', AIMED AT RETURNING GENERAL DATA TO THE STATE DEFINED IN THE FUNCTION ARGUMENT.'''
    
    df = margens_13.drop(columns = {'Obs: A partir de abril/04, passou a vigorar a nova sistemática de cobrança de ICMS de GLP em 11 estados (AM, AL, BA, CE, MA, RN, PA, PB, PE, RJ e SE), que consideram que parte do\rproduto é produzido a partir do gás natural.\rA parcela referente à bi-tributação de ICMS nestes estados está incluída na Margem Bruta de Distribuição.\r(1) A Resolução G SEFAZ n.o 13/08, de 22/09/08, incluiu o GLP acondicionado em recipientes transportáveis com capacidade de até 13 kg, destinado ao consumo doméstico, na lista\rde mercadorias integrantes da cesta básica. Dessa forma, sua alíquota de ICMS caiu de 17% para 1%, conforme Resolução G SEFAZ n.o 11/08, de 14/08/08. A Lei Estadual n.o 3.361,\rde 30/12/2008, em vigor a partir de 1o de janeiro de 2009, isentou do ICMS as operações internas com GLP destinado ao consumo doméstico e acondicionado em recipientes\rtransportáveis com capacidade de até 13 kg. A alínea "b" da Lei Complementar Estadual no 112, de 21/12/2012, em vigor desde 1/4/2013, alterou para 17% o valor da alíquota de\rICMS das operações internas com GLP.\r(2) Não foi possível calcular as margens brutas de distribuição e de revenda em função da ausência de preços de distribuição.\rNotas:\rPreços de Produtores:\rDesde janeiro de 2002, preços informados, semanalmente, pelos produtores e importadores, nos termos da Portaria ANP n.o 297, de 18/12/2001.\rEm contrapartida à retirada do subsídio aos preços de GLP, foi instituído, por meio do Decreto n.o 4.102, de 24/01/2002, o programa "Auxílio-Gás" no valor de R$ 7,50 / mês para\rfamílias de baixa renda, posteriormente unificado pelo programa "Bolsa-Família" por meio da Lei n.o 10.836, de 09 de janeiro de 2004.\rOs preços referem-se às vendas de GLP por botijão de 13 kg.\rICMS:\rAs alíquotas, as margens de valor agregado e os preços médios ponderados ao consumidor final (PMPF) são estabelecidos por Convênios CONFAZ e Atos COTEPE e variam de estado\rpara estado.\rPara formação do preço Brasil, o ICMS foi calculado com base nos valores de cada estado, ponderado pelos  respectivos volumes de comercialização.\rMargens de Distribuição e de Revenda:\rCalculadas com base nos dados semanais do Levantamento de Preços e de Margens de Comercialização de Combustíveis da ANP, regulamentado pela Portaria ANP n.o 202, de\r15/08/2000.\rOs preços de distribuição podem, eventualmente, contemplar valores relativos à descontos por pagamentos de duplicatas no vencimento e por cumprimento de metas de vendas.'})
    df = df.drop(columns = {'Unnamed: 6','Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5'})
    df = df.reset_index(drop=True).drop(index=[0,2,3,4,6,7,8,9,10,11,12])
    df = df.rename(columns = {'Unnamed: 0':'ESTADO','Unnamed: 1':'ICMS'	})
    df = df.reset_index(drop=True)
    df = df.iloc[38:49]
    df = df.drop(index = [38,39,40,42,43,44,45,46,47,48])
    df= df.rename(columns = {'ICMS':'ICMS em %'})
    df = df.replace('12%', estado).replace('ICMS','12%')
    df = df.set_index('ESTADO')
    
    return df

In [25]:
# NEW DATAFRAME TREATED (distrito_13):

distrito_13 = distrito_icms_13(margens_13,'DF')

In [26]:
# DEFINING THE FUNCTION FOR THE STATE THAT DO NOT REQUIRE SPECIAL TREATMENT IN THE MAIN DATAFRAME:

def piaui_icms_13(margens_13, estado): 
    
    '''THIS FUNCTION CARRIES OUT THE CLEANING TREATMENT OF THE DATAFRAME 'margens_13', AIMED AT RETURNING GENERAL DATA TO THE STATE DEFINED IN THE FUNCTION ARGUMENT.'''
    
    df = margens_13.drop(columns = {'Obs: A partir de abril/04, passou a vigorar a nova sistemática de cobrança de ICMS de GLP em 11 estados (AM, AL, BA, CE, MA, RN, PA, PB, PE, RJ e SE), que consideram que parte do\rproduto é produzido a partir do gás natural.\rA parcela referente à bi-tributação de ICMS nestes estados está incluída na Margem Bruta de Distribuição.\r(1) A Resolução G SEFAZ n.o 13/08, de 22/09/08, incluiu o GLP acondicionado em recipientes transportáveis com capacidade de até 13 kg, destinado ao consumo doméstico, na lista\rde mercadorias integrantes da cesta básica. Dessa forma, sua alíquota de ICMS caiu de 17% para 1%, conforme Resolução G SEFAZ n.o 11/08, de 14/08/08. A Lei Estadual n.o 3.361,\rde 30/12/2008, em vigor a partir de 1o de janeiro de 2009, isentou do ICMS as operações internas com GLP destinado ao consumo doméstico e acondicionado em recipientes\rtransportáveis com capacidade de até 13 kg. A alínea "b" da Lei Complementar Estadual no 112, de 21/12/2012, em vigor desde 1/4/2013, alterou para 17% o valor da alíquota de\rICMS das operações internas com GLP.\r(2) Não foi possível calcular as margens brutas de distribuição e de revenda em função da ausência de preços de distribuição.\rNotas:\rPreços de Produtores:\rDesde janeiro de 2002, preços informados, semanalmente, pelos produtores e importadores, nos termos da Portaria ANP n.o 297, de 18/12/2001.\rEm contrapartida à retirada do subsídio aos preços de GLP, foi instituído, por meio do Decreto n.o 4.102, de 24/01/2002, o programa "Auxílio-Gás" no valor de R$ 7,50 / mês para\rfamílias de baixa renda, posteriormente unificado pelo programa "Bolsa-Família" por meio da Lei n.o 10.836, de 09 de janeiro de 2004.\rOs preços referem-se às vendas de GLP por botijão de 13 kg.\rICMS:\rAs alíquotas, as margens de valor agregado e os preços médios ponderados ao consumidor final (PMPF) são estabelecidos por Convênios CONFAZ e Atos COTEPE e variam de estado\rpara estado.\rPara formação do preço Brasil, o ICMS foi calculado com base nos valores de cada estado, ponderado pelos  respectivos volumes de comercialização.\rMargens de Distribuição e de Revenda:\rCalculadas com base nos dados semanais do Levantamento de Preços e de Margens de Comercialização de Combustíveis da ANP, regulamentado pela Portaria ANP n.o 202, de\r15/08/2000.\rOs preços de distribuição podem, eventualmente, contemplar valores relativos à descontos por pagamentos de duplicatas no vencimento e por cumprimento de metas de vendas.'})
    df = df.drop(columns = {'Unnamed: 6','Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5'})
    df = df.reset_index(drop=True).drop(index=[0,2,3,4,6,7,8,9,10,11,12])
    df = df.rename(columns = {'Unnamed: 0':'ESTADO','Unnamed: 1':'ICMS'	})
    df = df.reset_index(drop=True)
    df = df.iloc[69:80]
    df = df.drop(index = [69,70,71,73,74,75,76,77,78,79])
    df= df.rename(columns = {'ICMS':'ICMS em %'})
    df = df.replace('17%', estado).replace('ICMS','17%')
    df = df.set_index('ESTADO')
    
    return df

In [27]:
# NEW DATAFRAME TREATED (pi_13):

pi_13 = piaui_icms_13(margens_13,'PI')

In [28]:
# DEFINING THE FUNCTION FOR THE STATE THAT DO NOT REQUIRE SPECIAL TREATMENT IN THE MAIN DATAFRAME:

def espirito_icms_13(margens_13, estado): 
    
    '''THIS FUNCTION CARRIES OUT THE CLEANING TREATMENT OF THE DATAFRAME 'margens_13', AIMED AT RETURNING GENERAL DATA TO THE STATE DEFINED IN THE FUNCTION ARGUMENT.'''
    
    df = margens_13.drop(columns = {'Obs: A partir de abril/04, passou a vigorar a nova sistemática de cobrança de ICMS de GLP em 11 estados (AM, AL, BA, CE, MA, RN, PA, PB, PE, RJ e SE), que consideram que parte do\rproduto é produzido a partir do gás natural.\rA parcela referente à bi-tributação de ICMS nestes estados está incluída na Margem Bruta de Distribuição.\r(1) A Resolução G SEFAZ n.o 13/08, de 22/09/08, incluiu o GLP acondicionado em recipientes transportáveis com capacidade de até 13 kg, destinado ao consumo doméstico, na lista\rde mercadorias integrantes da cesta básica. Dessa forma, sua alíquota de ICMS caiu de 17% para 1%, conforme Resolução G SEFAZ n.o 11/08, de 14/08/08. A Lei Estadual n.o 3.361,\rde 30/12/2008, em vigor a partir de 1o de janeiro de 2009, isentou do ICMS as operações internas com GLP destinado ao consumo doméstico e acondicionado em recipientes\rtransportáveis com capacidade de até 13 kg. A alínea "b" da Lei Complementar Estadual no 112, de 21/12/2012, em vigor desde 1/4/2013, alterou para 17% o valor da alíquota de\rICMS das operações internas com GLP.\r(2) Não foi possível calcular as margens brutas de distribuição e de revenda em função da ausência de preços de distribuição.\rNotas:\rPreços de Produtores:\rDesde janeiro de 2002, preços informados, semanalmente, pelos produtores e importadores, nos termos da Portaria ANP n.o 297, de 18/12/2001.\rEm contrapartida à retirada do subsídio aos preços de GLP, foi instituído, por meio do Decreto n.o 4.102, de 24/01/2002, o programa "Auxílio-Gás" no valor de R$ 7,50 / mês para\rfamílias de baixa renda, posteriormente unificado pelo programa "Bolsa-Família" por meio da Lei n.o 10.836, de 09 de janeiro de 2004.\rOs preços referem-se às vendas de GLP por botijão de 13 kg.\rICMS:\rAs alíquotas, as margens de valor agregado e os preços médios ponderados ao consumidor final (PMPF) são estabelecidos por Convênios CONFAZ e Atos COTEPE e variam de estado\rpara estado.\rPara formação do preço Brasil, o ICMS foi calculado com base nos valores de cada estado, ponderado pelos  respectivos volumes de comercialização.\rMargens de Distribuição e de Revenda:\rCalculadas com base nos dados semanais do Levantamento de Preços e de Margens de Comercialização de Combustíveis da ANP, regulamentado pela Portaria ANP n.o 202, de\r15/08/2000.\rOs preços de distribuição podem, eventualmente, contemplar valores relativos à descontos por pagamentos de duplicatas no vencimento e por cumprimento de metas de vendas.'})
    df = df.drop(columns = {'Unnamed: 6','Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5'})
    df = df.reset_index(drop=True).drop(index=[0,2,3,4,6,7,8,9,10,11,12])
    df = df.rename(columns = {'Unnamed: 0':'ESTADO','Unnamed: 1':'ICMS'	})
    df = df.reset_index(drop=True)
    df = df.iloc[49:57]
    df = df.drop(index = [49,50,51,53,54,55,56])
    df= df.rename(columns = {'ICMS':'ICMS em %'})
    df = df.replace('17%', estado).replace('ICMS','17%')
    df = df.set_index('ESTADO')
    
    return df

In [29]:
# NEW DATAFRAME TRATED (es_13):

es_13 = espirito_icms_13(margens_13,'ES')

In [30]:
# DEFINING THE FUNCTION FOR THE STATE THAT DO NOT REQUIRE SPECIAL TREATMENT IN THE MAIN DATAFRAME:

def rj_icms_13(margens_13, estado): 
    
    '''THIS FUNCTION CARRIES OUT THE CLEANING TREATMENT OF THE DATAFRAME 'margens_13', AIMED AT RETURNING GENERAL DATA TO THE STATE DEFINED IN THE FUNCTION ARGUMENT.'''
    
    df = margens_13.drop(columns = {'Obs: A partir de abril/04, passou a vigorar a nova sistemática de cobrança de ICMS de GLP em 11 estados (AM, AL, BA, CE, MA, RN, PA, PB, PE, RJ e SE), que consideram que parte do\rproduto é produzido a partir do gás natural.\rA parcela referente à bi-tributação de ICMS nestes estados está incluída na Margem Bruta de Distribuição.\r(1) A Resolução G SEFAZ n.o 13/08, de 22/09/08, incluiu o GLP acondicionado em recipientes transportáveis com capacidade de até 13 kg, destinado ao consumo doméstico, na lista\rde mercadorias integrantes da cesta básica. Dessa forma, sua alíquota de ICMS caiu de 17% para 1%, conforme Resolução G SEFAZ n.o 11/08, de 14/08/08. A Lei Estadual n.o 3.361,\rde 30/12/2008, em vigor a partir de 1o de janeiro de 2009, isentou do ICMS as operações internas com GLP destinado ao consumo doméstico e acondicionado em recipientes\rtransportáveis com capacidade de até 13 kg. A alínea "b" da Lei Complementar Estadual no 112, de 21/12/2012, em vigor desde 1/4/2013, alterou para 17% o valor da alíquota de\rICMS das operações internas com GLP.\r(2) Não foi possível calcular as margens brutas de distribuição e de revenda em função da ausência de preços de distribuição.\rNotas:\rPreços de Produtores:\rDesde janeiro de 2002, preços informados, semanalmente, pelos produtores e importadores, nos termos da Portaria ANP n.o 297, de 18/12/2001.\rEm contrapartida à retirada do subsídio aos preços de GLP, foi instituído, por meio do Decreto n.o 4.102, de 24/01/2002, o programa "Auxílio-Gás" no valor de R$ 7,50 / mês para\rfamílias de baixa renda, posteriormente unificado pelo programa "Bolsa-Família" por meio da Lei n.o 10.836, de 09 de janeiro de 2004.\rOs preços referem-se às vendas de GLP por botijão de 13 kg.\rICMS:\rAs alíquotas, as margens de valor agregado e os preços médios ponderados ao consumidor final (PMPF) são estabelecidos por Convênios CONFAZ e Atos COTEPE e variam de estado\rpara estado.\rPara formação do preço Brasil, o ICMS foi calculado com base nos valores de cada estado, ponderado pelos  respectivos volumes de comercialização.\rMargens de Distribuição e de Revenda:\rCalculadas com base nos dados semanais do Levantamento de Preços e de Margens de Comercialização de Combustíveis da ANP, regulamentado pela Portaria ANP n.o 202, de\r15/08/2000.\rOs preços de distribuição podem, eventualmente, contemplar valores relativos à descontos por pagamentos de duplicatas no vencimento e por cumprimento de metas de vendas.'})
    df = df.drop(columns = {'Unnamed: 6','Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5'})
    df = df.reset_index(drop=True).drop(index=[0,2,3,4,6,7,8,9,10,11,12])
    df = df.rename(columns = {'Unnamed: 0':'ESTADO','Unnamed: 1':'ICMS'	})
    df = df.reset_index(drop=True)
    df = df.iloc[80:91]
    df = df.drop(index = [80,81,82,84,85,86,87,88,89,90])
    df= df.rename(columns = {'ICMS':'ICMS em %'})
    df = df.replace('12%', estado).replace('ICMS','12%')
    df = df.set_index('ESTADO')
    
    return df

In [31]:
# NEW DATAFRAME TREATED (rj_13):

rj_13 = rj_icms_13(margens_13,'RJ')

In [32]:
# DEFINING THE FUNCTION FOR THE STATE THAT DO NOT REQUIRE SPECIAL TREATMENT IN THE MAIN DATAFRAME:

def rn_icms_13(margens_13, estado): 
    
    '''THIS FUNCTION CARRIES OUT THE CLEANING TREATMENT OF THE DATAFRAME 'margens_13', AIMED AT RETURNING GENERAL DATA TO THE STATE DEFINED IN THE FUNCTION ARGUMENT.'''

    df = margens_13.drop(columns = {'Obs: A partir de abril/04, passou a vigorar a nova sistemática de cobrança de ICMS de GLP em 11 estados (AM, AL, BA, CE, MA, RN, PA, PB, PE, RJ e SE), que consideram que parte do\rproduto é produzido a partir do gás natural.\rA parcela referente à bi-tributação de ICMS nestes estados está incluída na Margem Bruta de Distribuição.\r(1) A Resolução G SEFAZ n.o 13/08, de 22/09/08, incluiu o GLP acondicionado em recipientes transportáveis com capacidade de até 13 kg, destinado ao consumo doméstico, na lista\rde mercadorias integrantes da cesta básica. Dessa forma, sua alíquota de ICMS caiu de 17% para 1%, conforme Resolução G SEFAZ n.o 11/08, de 14/08/08. A Lei Estadual n.o 3.361,\rde 30/12/2008, em vigor a partir de 1o de janeiro de 2009, isentou do ICMS as operações internas com GLP destinado ao consumo doméstico e acondicionado em recipientes\rtransportáveis com capacidade de até 13 kg. A alínea "b" da Lei Complementar Estadual no 112, de 21/12/2012, em vigor desde 1/4/2013, alterou para 17% o valor da alíquota de\rICMS das operações internas com GLP.\r(2) Não foi possível calcular as margens brutas de distribuição e de revenda em função da ausência de preços de distribuição.\rNotas:\rPreços de Produtores:\rDesde janeiro de 2002, preços informados, semanalmente, pelos produtores e importadores, nos termos da Portaria ANP n.o 297, de 18/12/2001.\rEm contrapartida à retirada do subsídio aos preços de GLP, foi instituído, por meio do Decreto n.o 4.102, de 24/01/2002, o programa "Auxílio-Gás" no valor de R$ 7,50 / mês para\rfamílias de baixa renda, posteriormente unificado pelo programa "Bolsa-Família" por meio da Lei n.o 10.836, de 09 de janeiro de 2004.\rOs preços referem-se às vendas de GLP por botijão de 13 kg.\rICMS:\rAs alíquotas, as margens de valor agregado e os preços médios ponderados ao consumidor final (PMPF) são estabelecidos por Convênios CONFAZ e Atos COTEPE e variam de estado\rpara estado.\rPara formação do preço Brasil, o ICMS foi calculado com base nos valores de cada estado, ponderado pelos  respectivos volumes de comercialização.\rMargens de Distribuição e de Revenda:\rCalculadas com base nos dados semanais do Levantamento de Preços e de Margens de Comercialização de Combustíveis da ANP, regulamentado pela Portaria ANP n.o 202, de\r15/08/2000.\rOs preços de distribuição podem, eventualmente, contemplar valores relativos à descontos por pagamentos de duplicatas no vencimento e por cumprimento de metas de vendas.'})
    df = df.drop(columns = {'Unnamed: 6','Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5'})
    df = df.reset_index(drop=True).drop(index=[0,2,3,4,6,7,8,9,10,11,12])
    df = df.rename(columns = {'Unnamed: 0':'ESTADO','Unnamed: 1':'ICMS'	})
    df = df.reset_index(drop=True)
    df = df.iloc[91:99]
    df = df.drop(index = [91,92,93,95,96,97,98])
    df= df.rename(columns = {'ICMS':'ICMS em %'})
    df = df.replace('do produtor)', estado).replace('ICMS','17%')
    df = df.set_index('ESTADO')
    
    return df

In [33]:
# NEW DATAFRAME TREATED (rn_13):

rn_13 = rn_icms_13(margens_13,'RN')

In [34]:
def limpar_meses_icms(df):
    '''THIS FUNCTION USES THE 'margins_13' DATAFRAME TO CLEAN AND PROCESS GENERAL DATA WITH THE PURPOSE OF EXTRACTING THE ICMS FROM EACH STATE.'''
    
# GENERAL TREATMENT:
    df = df.drop(columns = {'Obs: A partir de abril/04, passou a vigorar a nova sistemática de cobrança de ICMS de GLP em 11 estados (AM, AL, BA, CE, MA, RN, PA, PB, PE, RJ e SE), que consideram que parte do\rproduto é produzido a partir do gás natural.\rA parcela referente à bi-tributação de ICMS nestes estados está incluída na Margem Bruta de Distribuição.\r(1) A Resolução G SEFAZ n.o 13/08, de 22/09/08, incluiu o GLP acondicionado em recipientes transportáveis com capacidade de até 13 kg, destinado ao consumo doméstico, na lista\rde mercadorias integrantes da cesta básica. Dessa forma, sua alíquota de ICMS caiu de 17% para 1%, conforme Resolução G SEFAZ n.o 11/08, de 14/08/08. A Lei Estadual n.o 3.361,\rde 30/12/2008, em vigor a partir de 1o de janeiro de 2009, isentou do ICMS as operações internas com GLP destinado ao consumo doméstico e acondicionado em recipientes\rtransportáveis com capacidade de até 13 kg. A alínea "b" da Lei Complementar Estadual no 112, de 21/12/2012, em vigor desde 1/4/2013, alterou para 17% o valor da alíquota de\rICMS das operações internas com GLP.\r(2) Não foi possível calcular as margens brutas de distribuição e de revenda em função da ausência de preços de distribuição.\rNotas:\rPreços de Produtores:\rDesde janeiro de 2002, preços informados, semanalmente, pelos produtores e importadores, nos termos da Portaria ANP n.o 297, de 18/12/2001.\rEm contrapartida à retirada do subsídio aos preços de GLP, foi instituído, por meio do Decreto n.o 4.102, de 24/01/2002, o programa "Auxílio-Gás" no valor de R$ 7,50 / mês para\rfamílias de baixa renda, posteriormente unificado pelo programa "Bolsa-Família" por meio da Lei n.o 10.836, de 09 de janeiro de 2004.\rOs preços referem-se às vendas de GLP por botijão de 13 kg.\rICMS:\rAs alíquotas, as margens de valor agregado e os preços médios ponderados ao consumidor final (PMPF) são estabelecidos por Convênios CONFAZ e Atos COTEPE e variam de estado\rpara estado.\rPara formação do preço Brasil, o ICMS foi calculado com base nos valores de cada estado, ponderado pelos  respectivos volumes de comercialização.\rMargens de Distribuição e de Revenda:\rCalculadas com base nos dados semanais do Levantamento de Preços e de Margens de Comercialização de Combustíveis da ANP, regulamentado pela Portaria ANP n.o 202, de\r15/08/2000.\rOs preços de distribuição podem, eventualmente, contemplar valores relativos à descontos por pagamentos de duplicatas no vencimento e por cumprimento de metas de vendas.'})
    df = df.drop(columns = {'Unnamed: 6','Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5'})
    df = df.reset_index(drop=True).drop(index=[0,2,3,4,6,7,8,9,10,11,12])
    df = df.rename(columns = {'Unnamed: 0':'ESTADO','Unnamed: 1':'ICMS em %'})
    df = df.reset_index(drop=True)
    df = df.iloc[[21,22,23,57,58,59,60,61,62,63,64,65,99,100,101,102,103,104,105]]
    lista_estado = ['AP','AM','BA','GO','MA','MT','MS','MG','PA','PB','PR','PE','RS','RO','RR','SC','SP','SE','TO']
    lista_icms = ['12%','17%','12%','12%','17%','12%','12%','18%','17%','17%','12%','17%','12%','12%','17%','12%','12%','17%','12%']
    df['ICMS em %'] = lista_icms
    df['ESTADO'] = lista_estado

#'jan/13' COLUMN TREATMENT:

    df['jan/13'] = df['jan/13'].str.slice(start=33)
    df['jan/13'] = df['jan/13'].str.replace(r'%\r','',regex = True)
    df['jan/13'] = df['jan/13'].str.slice(start=0, stop=4)
    df['jan/13'] = df['jan/13'].str.replace('-\r17','-')
    df['jan/13'] = df['jan/13'].str.replace(',','.')

#'fev/13' COLUMN TREATMENT:

    df['fev/13'] = df['fev/13'].str.slice(start=33)
    df['fev/13'] = df['fev/13'].str.replace(r'%\r','',regex = True)
    df['fev/13'] = df['fev/13'].str.slice(start=0, stop=4)
    df['fev/13'] = df['fev/13'].str.replace('-\r21','-')
    df['fev/13'] = df['fev/13'].str.replace(',','.')


#'mar/13' COLUMN TREATMENT:

    df['mar/13'] = df['mar/13'].str.slice(start=33)
    df['mar/13'] = df['mar/13'].str.replace(r'%\r','',regex = True)
    df['mar/13'] = df['mar/13'].str.slice(start=0, stop=4)
    df['mar/13'] = df['mar/13'].str.replace('-\r22','-')
    df['mar/13'] = df['mar/13'].str.replace(',','.')


#'abr/13' COLUMN TREATMENT:

    df['abr/13'] = df['abr/13'].str.slice(start=33)
    df['abr/13'] = df['abr/13'].str.replace(r'%\r','',regex = True)
    df['abr/13'] = df['abr/13'].str.slice(start=0, stop=4)
    df['abr/13'] = df['abr/13'].str.replace(',','.')


#'mai/13' COLUMN TREATMENT:

    df['mai/13'] = df['mai/13'].str.slice(start=33)
    df['mai/13'] = df['mai/13'].str.replace(r'%\r','',regex = True)
    df['mai/13'] = df['mai/13'].str.slice(start=0, stop=4)
    df['mai/13'] = df['mai/13'].str.replace(',','.')


#'jun/13' COLUMN TREATMENT:

    df['jun/13'] = df['jun/13'].str.slice(start=33)
    df['jun/13'] = df['jun/13'].str.replace(r'%\r','',regex = True)
    df['jun/13'] = df['jun/13'].str.slice(start=0, stop=4)
    df['jun/13'] = df['jun/13'].str.replace(',','.')


#'jul/13' COLUMN TREATMENT:

    df['jul/13'] = df['jul/13'].str.slice(start=33)
    df['jul/13'] = df['jul/13'].str.replace(r'%\r','',regex = True)
    df['jul/13'] = df['jul/13'].str.slice(start=0, stop=4)
    df['jul/13'] = df['jul/13'].str.replace(',','.')


#'ago/13' COLUMN TREATMENT:

    df['ago/13'] = df['ago/13'].str.slice(start=33)
    df['ago/13'] = df['ago/13'].str.replace(r'%\r','',regex = True)
    df['ago/13'] = df['ago/13'].str.slice(start=0, stop=4)
    df['ago/13'] = df['ago/13'].str.replace(',','.')


#'set/13' COLUMN TREATMENT:

    df['set/13'] = df['set/13'].str.slice(start=33)
    df['set/13'] = df['set/13'].str.replace(r'%\r','',regex = True)
    df['set/13'] = df['set/13'].str.slice(start=0, stop=4)
    df['set/13'] = df['set/13'].str.replace(',','.')


#'out/13' COLUMN TREATMENT:

    df['out/13'] = df['out/13'].str.slice(start=33)
    df['out/13'] = df['out/13'].str.replace(r'%\r','',regex = True)
    df['out/13'] = df['out/13'].str.slice(start=0, stop=4)
    df['out/13'] = df['out/13'].str.replace(',','.')


#'nov/13' COLUMN TREATMENT:

    df['nov/13'] = df['nov/13'].str.slice(start=33)
    df['nov/13'] = df['nov/13'].str.replace(r'%\r','',regex = True)
    df['nov/13'] = df['nov/13'].str.slice(start=0, stop=4)
    df['nov/13'] = df['nov/13'].str.replace(',','.')


#'dez/13' COLUMN TREATMENT:

    df['dez/13'] = df['dez/13'].str.slice(start=33)
    df['dez/13'] = df['dez/13'].str.replace(r'%\r','',regex = True)
    df['dez/13'] = df['dez/13'].str.slice(start=0, stop=4)
    df['dez/13'] = df['dez/13'].str.replace(',','.')

# TRANSFORMING COLUMN 'ESTADO' IN INDEX:
    df = df.set_index('ESTADO')

    return df

In [35]:
# NEW DATAFRAME TREATED (faltantes_icms):

faltantes_icms = limpar_meses_icms(margens_13)

### CONCATENATING THE ICMS TABLES BY STATE IN A NEW DATAFRAME

In [36]:
# NEW DATAFRAME TREATED (estados_icms_2013):

estados_icms_2013 = pd.concat([faltantes_icms, ac_13, al_13, ce_13, distrito_13, pi_13, es_13, rj_13, rn_13 ], axis=0).reset_index()

### DATAFRAME complemento_icms

In [37]:
def limpar_complemento(df):
    '''THIS FUNCTION USES THE 'complemento_icms' DATAFRAME TO CLEAN AND PROCESS GENERAL DATA WITH THE PURPOSE OF EXTRACTING THE ICMS FROM EACH STATE.'''
    df = df.rename(columns = {'Column1':'ESTADO','Column2':'ICMS em %'})
    df = df.reset_index(drop=True)
    df = df.drop(index=[6,7,8])
    jl = ['10.92', '20.73', '16.58', '14.68', '8.86', '13.68', '15.40']
    df['jan/23'] = jl
    jf = ['12.01', '20.86', '16.58', '14.80', '8.86', '14.76', '15.60']
    df['fev/23'] = jf
    pc =['12%','17%','18%','18%','12%', '17%', '18%']
    df['ICMS em %'] = pc
    df = df.set_index('ESTADO')
    
    return df

In [38]:
# NEW DATAFRAME TREATED (estados_complemento):

estados_complemento = limpar_complemento(complemento_icms)

### DATAFRAME margens_23

In [39]:
# DEFINING THE FUNCTION THAT EXTRACTS BRAZIL ONLY FROM THE GROUP:
def brasil_icms_23(df):
    
    '''THIS FUNCTION CARRIES OUT THE CLEANING TREATMENT OF THE 'margens_23' DATAFRAME, AIMED AT RETURNING GENERAL DATA FOR BRAZIL.'''

# GENERAL TREATMENT:

    df = df.iloc[0:1]
    df = df.rename(columns = {'Unnamed: 1': 'ICMS em %', 'Unnamed: 0':'ESTADO'})
    df['ICMS em %']= df['ICMS em %'].str.replace('Preço de Realização do Produtor\rCIDE\rPIS/COFINS\rPreço do Produtor s/ ICMS c/ CIDE/PIS/COFINS\rICMS\rMargem Bruta de Distribuição\rPreço de Distribuição\rMargem Bruta de Revenda\rPreço Final ao Consumidor', '-')
    df = df.set_index('ESTADO')

#'jan/23' COLUMN TREATMENT:

    df['jan/23'] = df['jan/23'].str.slice(start=16)
    df['jan/23'] = df['jan/23'].str.slice(start=0, stop=5)
    df['jan/23'] = df['jan/23'].str.replace(',','.')

#'fev/23' COLUMN TREATMENT:

    df['fev/23'] = df['fev/23'].str.slice(start=16)
    df['fev/23'] = df['fev/23'].str.slice(start=0, stop=5)
    df['fev/23'] = df['fev/23'].str.replace(',','.')

    return df

In [40]:
# NEW DATAFRAME TREATED (br_icms_23):

br_icms_23 = brasil_icms_23(margens_23).reset_index()

In [41]:
def limpar_margens_23(df):
   
    '''THIS FUNCTION USES THE 'margins_23' DATAFRAME TO CLEAN AND PROCESS GENERAL DATA WITH THE PURPOSE OF EXTRACTING THE ICMS FROM EACH STATE.'''

# GENERAL TREATMENT:
    
    df = df.drop(index=[0])
    df = df.rename(columns = {'Unnamed: 1': 'ICMS em %', 'Unnamed: 0':'ESTADO'})
    df['ICMS em %']= df['ICMS em %'].str.replace('Preço de Realização do Produtor\rCIDE\rPIS/COFINS\rPreço do Produtor s/ ICMS c/ CIDE/PIS/COFINS\rICMS\rMargem Bruta de Distribuição\rPreço de Distribuição\rMargem Bruta de Revenda\rPreço Final ao Consumidor', '-')
    lista_23 = ['AC','AL','AP','BA','CE','DF','G0','MA','MT','MG','PA','PB','PE','PI','RJ','RS','RO','RR','SP','SE']
    icms_lista_23 = ['17%','18%','18%','12%','18%','12%','12%','14%','12%','18%','17%','18%','18%','12%','12%','12%','12%','12%','12%','12%']
    df['ESTADO'] = lista_23
    df['ICMS em %'] = icms_lista_23 
    df = df.set_index('ESTADO')


#'jan/23' COLUMN TREATMENT:

    df['jan/23'] = df['jan/23'].str.slice(start=16, stop=27)
    lj = ['16,09','13,88','16,64','9,19','14,32','13,31','11,62','11,52','13,37','14,84','14,89','14,86','13,55','10,69','9,75','9,67','15,07','12,81','13,60','10,57']
    df['jan/23'] = lj
    df['jan/23'] = df['jan/23'].str.replace(',','.')


#'fev/23' COLUMN TREATMENT:
    
    df['fev/23'] = df['fev/23'].str.slice(start=16, stop=27)
    lf = ['16,22','13,88','16,78','19,37','15,02','13,44','13,37','13,49','14,45','14,97','15,02','15,91','13,67','10,69','10,91','9,75','14,63','14,13','12,87','9,74']
    df['fev/23'] = lf
    df['fev/23'] = df['fev/23'].str.replace(',','.')


    return df

In [42]:
# NEW DATAFRAME TREATED (br_icms_23):


estados_a =limpar_margens_23(margens_23)

### CONCATENATING THE ICMS TABLES BY STATE IN A NEW DATAFRAME

In [43]:
# NEW DATAFRAME TREATED (estados_icms_2023):

estados_icms_2023 = pd.concat([estados_a, estados_complemento], axis=0)
estados_icms_2023 = estados_icms_2023. reset_index()


In [47]:
glp_br

,DATA,PRODUTO,NÚMERO DE POSTOS PESQUISADOS,UNIDADE DE MEDIDA,PREÇO MÉDIO REVENDA,PREÇO MÍNIMO REVENDA,PREÇO MÁXIMO REVENDA
0,2013-01-01,GLP,8052,13Kg,40.113,26.00,62.0
1,2013-01-01,GLP,8053,13Kg,40.143,26.00,62.0
2,2013-01-01,GLP,8055,13Kg,40.104,27.00,62.0
3,2013-01-01,GLP,8029,13Kg,40.233,27.00,62.0
4,2013-02-01,GLP,8035,13Kg,40.222,26.00,62.0
...,...,...,...,...,...,...,...
520,2023-03-01,GLP,4137,13Kg,107.500,70.00,155.0
521,2023-03-01,GLP,4018,13Kg,107.500,70.00,155.0
522,2023-03-01,GLP,3877,13Kg,107.690,74.99,155.0
523,2023-03-01,GLP,3936,13Kg,107.520,74.99,155.0


## CREATING THE DATABASE IN MYSQL AND EXPORTING TABLES

In [44]:
from dotenv import load_dotenv
import sqlalchemy as sa
load_dotenv(r'C:\Users\pedro\OneDrive\Área de Trabalho\GAS\mysql.env')
url_banco = '127.0.0.1'
nome_db = 'gas'
usuario = os.getenv('usuario')
senha = os.getenv('senha')
conn_str = f"mysql+pymysql://{usuario}:{senha}@{url_banco}/{nome_db}"
engine = sa.create_engine(conn_str)

glp_br.to_sql('glp_br', engine, index = False, if_exists = 'replace')
glp_regioes.to_sql('glp_regioes', engine, index = False, if_exists = 'replace')
brasil_icms_13.to_sql('brasil_icms_2013', engine, index = False, if_exists = 'replace')
estados_icms_2013.to_sql('estados_icms_2013', engine, index = False, if_exists = 'replace')
br_icms_23.to_sql('br_icms_23', engine, index = False, if_exists = 'replace')
estados_icms_2023.to_sql('estados_icms_2023', engine, index = False, if_exists = 'replace')
salario_minimo.to_sql('salario_minimo', engine, index = False, if_exists = 'replace')

12

## CONCLUSIONS OBTAINED THROUGH THE GRAPHIC ANALYSIS IN THE TABLEAU

#### Based on the insights presented about GLP, we can conclude that there is a large discrepancy in the collection of ICMS in the different Brazilian states, with Minas Gerais being the state that has the most discrepant rate from the others. While some states have increased their taxes over the years, Minas Gerais has remained at the same rate. Despite this, in 2013, the state of Acre charged the highest added value in the price of LPG, while over the 10 years studied, the state that charged the most was Mato Grosso and the cheapest was Bahia. The Northeast region stood out as the most expensive in Brazil in relation to the price of LPG. The hypothesis that cooking gas represented a large margin of the minimum wage was refuted, and over the 10 years, it represented a margin of 6 to 9%, always readjusted according to the annual readjustment of the minimum wage. However, there was a significant increase of 8 to 9% in 2021 and 2022, which indicates the need for continuous monitoring to verify the stabilization or depreciation of rates in the coming years.